# RUSSIA-UKRAINE NTL ANALYSIS (WORK IN PROGRESS)


#INITIALISATION

In [ ]:
using Pkg
Pkg.add(["GeoJSON", "NighttimeLights", "Rasters", "CSV", "Dates", "DataFrames", "GeoDataFrames"])

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
   Installed LRUCache ────────────────── v1.6.2
   Installed GDAL_jll ────────────────── v303.1100.300+0
   Installed ImageIO ─────────────────── v0.6.8
   Installed ImageSegmentation ───────── v1.8.1
   Installed TiledIteration ──────────── v0.3.1
   Installed Accessors ───────────────── v0.1.42
   Installed TiffImages ──────────────── v0.10.2
   Installed DimensionalData ─────────── v0.27.9
   Installed ProgressMeter ───────────── v1.11.0
   Installed GeoJSON ─────────────────── v0.8.3
   Installed Qhull_jll ───────────────── v10008.0.1004+0
   Installed ImageMagick ─────────────── v1.4.2
   Installed NearestNeighbors ────────── v0.4.22
   Installed Roots ───────────────────── v2.2.10
   Installed RegionTrees ─────────────── v0.3.2
   Installed DBFTables ───────────────── v1.2.7
   Installed GLM ─────────────────────── v1.9.0
   Installed GeoInterfaceMakie ───────── v0.1.9
   Installed mupars

In [ ]:
;git clone https://github.com/xKDR/Shedding-light-on-the-Russia-Ukraine-war.git

In [ ]:
## Fetching the raster files
## This will take around 5 mins

run(`apt-get install git-lfs -y`)
run(`git lfs install`)
cd("Shedding-light-on-the-Russia-Ukraine-war")
run(`git lfs pull`)

In [ ]:
using GeoJSON
using GeoDataFrames
using DataFrames
using NighttimeLights
using Rasters
using CSV
using Dates

In [ ]:
# Define path constants
const FIGURES_DIRECTORY = "figures"
const TABLES_DIRECTORY = "tables"
const VECTOR_DIRECTORY = "data/vector"
const TABULAR_DIRECTORY = "data/tabular"
const RADIANCE_RASTER_DIRECTORY = "/content/Shedding-light-on-the-Russia-Ukraine-war/data/raster/rad_cropped"
const CFOBS_RASTER_DIRECTORY = "/content/Shedding-light-on-the-Russia-Ukraine-war/data/raster/cf_cropped"

In [ ]:
const START_DATE = Date(2014, 1)
const END_DATE = Date(2016, 1)

In [ ]:
function read_vector(filename)
# Use GeoJSON.read, which expects an IO
    geo = open(filename) do f
        GeoJSON.read(f)
    end

    # Now extract features
    features = geo.features

    # Build DataFrame with properties + geometry
    return DataFrame([merge(Dict(f.properties), Dict(:geometry => f.geometry)) for f in features])
end

#PROCESSING

In [15]:
function safe_centre_of_mass(slice::AbstractArray, state_name::String)
    s = sum(skipmissing(slice))
    if ismissing(s) || isnan(s) || s == 0.0f0
        return [NaN, NaN]
    end
    try
        return NighttimeLights.centre_of_mass(slice)
    catch e
        if isa(e, InexactError)
            println("Warning: CoM calculation failed for '$state_name'. Returning [NaN, NaN].")
            return [NaN, NaN]
        else
            rethrow(e)
        end
    end
end

safe_centre_of_mass (generic function with 1 method)

In [16]:
import Base.Filesystem: basename

"""
Extracts Date from filename.
Matches either `YYYY-MM-DD` or `YYYY-MM-DDTHH:MM:SS`.
"""
function file_to_date(f::AbstractString)
    b = basename(f)
    m = match(r"\d{4}-\d{2}-\d{2}(?:T\d{2}:\d{2}:\d{2})?", b)
    m === nothing && error("Cannot parse date from filename: $f")
    s = m.match
    if occursin('T', s)
        return Date(DateTime(s, dateformat"yyyy-mm-ddTHH:MM:SS"))
    else
        return Date(s, dateformat"yyyy-mm-dd")
    end
end


"""
Reads rasters between `start_date` and `end_date`
and returns cropped datacubes for radiance and cloud-free observations.
"""
function read_region_as_datacubes(region; start_date::Date, end_date::Date,
    radiance_path=RADIANCE_RASTER_DIRECTORY, cf_path=CFOBS_RASTER_DIRECTORY)

    # all .tif files
    rad_all = sort(joinpath.(radiance_path, filter(f -> endswith(f, ".tif"), readdir(radiance_path))))
    cf_all  = sort(joinpath.(cf_path, filter(f -> endswith(f, ".tif"), readdir(cf_path))))


    # restrict to date window
    rad_inwin = filter(f -> (d = file_to_date(f); start_date <= d <= end_date), rad_all)
    cf_inwin  = filter(f -> (d = file_to_date(f); start_date <= d <= end_date), cf_all)

    # sort by extracted date
    rad_inwin = rad_inwin[sortperm(file_to_date.(rad_inwin))]
    cf_inwin  = cf_inwin[sortperm(file_to_date.(cf_inwin))]

    # build raster lists
# build raster lists, now fully in memory
    rad_rasters = [Rasters.read(crop(Raster(f; lazy=true), to=region)) for f in rad_inwin]
    cf_rasters  = [Rasters.read(crop(Raster(f; lazy=true), to=region)) for f in cf_inwin]


    # raster series with actual dates
    rad_series = RasterSeries(rad_rasters, Ti(file_to_date.(rad_inwin)))
    cf_series  = RasterSeries(cf_rasters,  Ti(file_to_date.(cf_inwin)))

    # convert to datacubes
    rad_datacube = Rasters.combine(rad_series, Ti)
    cf_datacube  = Rasters.combine(cf_series, Ti)
    return rad_datacube, cf_datacube
end


read_region_as_datacubes

In [17]:
function process_state(state_row::DataFrameRow)
    state_name = state_row.NAME_1
    state_geometry = state_row.geometry
    @show state_name
    @show state_geometry
    radiance, cfobs = read_region_as_datacubes(state_geometry;
        start_date=START_DATE, end_date=END_DATE,
        radiance_path=RADIANCE_RASTER_DIRECTORY,
        cf_path=CFOBS_RASTER_DIRECTORY)


    print("file read correctly")

    radiance = clean_complete(radiance, cfobs; bgthreshold=4)
    cfobs = nothing # Release memory

    radiance = Raster(Float32.(radiance); missingval=missing)
    radiance[radiance .> 100.0f0] .= 0.0f0 # Cap high radiance values
    radiance = mask(radiance; with=state_geometry)

    dates = Date.(dims(radiance, Ti).val)
    num_timesteps = length(dates)

    if num_timesteps == 0
        println("Warning: No valid data for '$state_name' after cleaning.")
        return DataFrame()
    end

    agg_radiance = [sum(skipmissing(view(radiance, Ti(i)))) for i in 1:num_timesteps]
    com = [safe_centre_of_mass(view(radiance, Ti(i)), state_name) for i in 1:num_timesteps]

    return DataFrame(; dates, aggregate_radiance=agg_radiance, com, state_name)
end

process_state (generic function with 1 method)

##RUSSIA

In [ ]:
russia_state_split = read_vector("/content/Shedding-light-on-the-Russia-Ukraine-war/data/vector/russia_processing.geojson")

In [ ]:
radiance, cfobs = read_region_as_datacubes(
    russia_state_split.geometry[1];
    start_date=START_DATE,
    end_date=END_DATE,
    radiance_path=RADIANCE_RASTER_DIRECTORY,
    cf_path=CFOBS_RASTER_DIRECTORY
)

In [ ]:
radiance, cfobs = read_region_as_datacubes(russia_state_split[1, :].geometry;
    start_date=START_DATE, end_date=END_DATE,
    radiance_path=RADIANCE_RASTER_DIRECTORY,
    cf_path=CFOBS_RASTER_DIRECTORY)

In [ ]:
all_states_russia = DataFrame()

for i in 1:nrow(russia_state_split)
    state_row = russia_state_split[i, :]
    state_name = state_row.NAME_1
    println("--- Processing state $i/$(nrow(russia_state_split)): $state_name ---")

    try
        state_df = process_state(state_row)
        if !isempty(state_df)
            append!(all_states_russia, state_df; cols=:union)
        end
    catch e
        println("ERROR: Failed to process state '$state_name'. Skipping. Error: $e")
    end
    GC.gc()
end

println("--- Processing Complete ---")

# Final aggregation
if !isempty(all_states_russia)
    println("Aggregating results...")
    grouped = groupby(all_states_russia, :dates)
    aggregate_df = DataFrames.combine(grouped, :aggregate_radiance => sum => :AggregateRadiance)
    sort!(aggregate_df, :dates)

    # only write once if you want
    # CSV.write(AGGREGATE_FILE, aggregate_df)
    println("Aggregation complete, DataFrame in memory.")
else
    println("No data processed, skipping aggregation.")
end

println("Script finished.")


In [ ]:
all_states_russia

##UKRAINE

In [18]:
ukraine_state = read_vector("/content/Shedding-light-on-the-Russia-Ukraine-war/data/vector/ukraine.geojson")

Row,geometry,NAME_1,NL_NAME_1,GID_0,VARNAME_1,HASC_1,ISO_1,TYPE_1,GID_1,COUNTRY,CC_1,ENGTYPE_1
,Abstract…,String,String,String,String,String,String,String,String,String,String,String
1,2D Polygon,?,?,UKR,?,?,NA,?,?,Ukraine,NA,?
2,2D Polygon,Cherkasy,Черкаська,UKR,Cherkas'ka Oblast'|Cherkasskaya,UA.CK,NA,Oblast',UKR.1_1,Ukraine,NA,Region
3,2D Polygon,Chernihiv,Чернігівська,UKR,Chernigov|Tschernigow,UA.CH,NA,Oblast',UKR.2_1,Ukraine,NA,Region
4,2D Polygon,Chernivtsi,Чернівецька,UKR,Chernivets'ka Oblast'|Chernovits,UA.CV,NA,Oblast',UKR.3_1,Ukraine,NA,Region
5,2D MultiPolygon,Crimea,Крим,UKR,Crimée|Criméia|Krim|Krymskaya Re,UA.KR,NA,Autonomous Republic,UKR.4_1,Ukraine,NA,Autonomous Republic
6,2D Polygon,Dnipropetrovs'k,Дніпропетро́вська,UKR,Dnipropetrovsk|Dniepropietrovsk|,UA.DP,NA,Oblast',UKR.5_1,Ukraine,NA,Region
7,2D MultiPolygon,Donets'k,Доне́цька,UKR,Donetsk|Donetskaya Oblast'|Donez,UA.DT,NA,Oblast',UKR.6_1,Ukraine,NA,Region
8,2D Polygon,Ivano-Frankivs'k,Івано-Франківська,UKR,Ivano-Frankovsk|Ivano-Frankovska,UA.IF,NA,Oblast',UKR.7_1,Ukraine,NA,Region
9,2D Polygon,Kharkiv,Харківська,UKR,Charkow|Jarkov|Karkov|Khar'kov,UA.KK,NA,Oblast',UKR.8_1,Ukraine,NA,Region


In [23]:
radiance, cfobs = read_region_as_datacubes(
    ukraine_state.geometry[1];
    start_date=START_DATE,
    end_date=END_DATE,
    radiance_path=RADIANCE_RASTER_DIRECTORY,
    cf_path=CFOBS_RASTER_DIRECTORY
)

LoadError: GDALError (CE_Failure, code 10):
	Pointer 'hDS' is NULL in 'GDALGetRasterCount'.



In [24]:
radiance, cfobs = read_region_as_datacubes(ukraine_state[1, :].geometry;
    start_date=START_DATE, end_date=END_DATE,
    radiance_path=RADIANCE_RASTER_DIRECTORY,
    cf_path=CFOBS_RASTER_DIRECTORY)

LoadError: GDALError (CE_Failure, code 10):
	Pointer 'hDS' is NULL in 'GDALGetRasterCount'.



In [21]:
all_states_ukraine = DataFrame()

for i in 1:nrow(ukraine_state)
    state_row = ukraine_state[i, :]
    state_name = state_row.NAME_1
    println("--- Processing state $i/$(nrow(ukraine_state)): $state_name ---")

    try
        state_df = process_state(state_row)
        if !isempty(state_df)
            append!(all_states_ukraine, state_df; cols=:union)
        end
    catch e
        println("ERROR: Failed to process state '$state_name'. Skipping. Error: $e")
    end
    GC.gc()
end

println("--- Processing Complete ---")

# Final aggregation
if !isempty(all_states_ukraine)
    println("Aggregating results...")
    grouped = groupby(all_states_ukraine, :dates)
    aggregate_df = DataFrames.combine(grouped, :aggregate_radiance => sum => :AggregateRadiance)
    sort!(aggregate_df, :dates)

    # only write once if you want
    # CSV.write(AGGREGATE_FILE, aggregate_df)
    println("Aggregation complete, DataFrame in memory.")
else
    println("No data processed, skipping aggregation.")
end

println("Script finished.")


--- Processing state 1/28: ? ---
state_name = "?"
state_geometry = 2D Polygonwith 1 sub-geometries
ERROR: Failed to process state '?'. Skipping. Error: GDAL.GDALError(GDAL.CE_Failure, 10, "Pointer 'hDS' is NULL in 'GDALGetRasterCount'.\n")
--- Processing state 2/28: Cherkasy ---
state_name = "Cherkasy"
state_geometry = 2D Polygonwith 1 sub-geometries
ERROR: Failed to process state 'Cherkasy'. Skipping. Error: GDAL.GDALError(GDAL.CE_Failure, 10, "Pointer 'hDS' is NULL in 'GDALGetRasterCount'.\n")
--- Processing state 3/28: Chernihiv ---
state_name = "Chernihiv"
state_geometry = 2D Polygonwith 1 sub-geometries
ERROR: Failed to process state 'Chernihiv'. Skipping. Error: GDAL.GDALError(GDAL.CE_Failure, 10, "Pointer 'hDS' is NULL in 'GDALGetRasterCount'.\n")
--- Processing state 4/28: Chernivtsi ---
state_name = "Chernivtsi"
state_geometry = 2D Polygonwith 1 sub-geometries
ERROR: Failed to process state 'Chernivtsi'. Skipping. Error: GDAL.GDALError(GDAL.CE_Failure, 10, "Pointer 'hDS' is NU

In [22]:
all_states_ukraine

0×0 DataFrame